In [ ]:
import os

#where it finds and list samples
directory = 'samples/'
files = os.listdir(directory)

import pandas as pd
import math

#load significant features
df = pd.read_csv("significant_features.csv")

#define m/z and rt windows
rt_window = float(input("Retention Time +/- (in minutes): "))
mz_window = float(input("m/z +/-: "))

#prepare for the result.csv file and start counter
data = {}
data_count = 0

for sample in files:    
    lb = []
    sample_path = directory + sample
    with open(sample_path, 'r') as f:
        for line in f.readlines():
            lb.append(line)
            if line.startswith("BEGIN IONS"):
                lb = []
                lb.append(line)
            elif line.startswith("END IONS"):
                #data recorded in list lb, now search for desired ion and write it to mgf file
                #collect individual sample ions data
                title = lb[1].split("=")[1]
                #print('title', title)
                mz_sample = float(title.split((",")[0])[0].split('(')[1][:-1])
                mz_sample_rounded = round(mz_sample, 1)
                mz_sample_one_dec = math.floor(mz_sample * 10 ** 1) / 10 ** 1
                #print('mzr', mzr)
                rt_sample = float(title.split(",")[2].split(' ')[1][:-3])
                rt_sample_rounded = round(rt_sample, 1)
                #print('rt', rt)
                precursor_ion_intensity = lb[4].split("=")[1].split('\t')[1][:-1]
                #print(precursor_ion_intensity)
                energy = title.split(",")[1].strip()
                for i in range(len(df['labels'])):
                    #collect feature data
                    label = df['labels'][i]
                    #print(label)
                    rt_feature = float(df['rt_values'][i])
                    rt_feature_rounded = round(rt_feature, 1)
                    rt_feature_one_dec = math.floor(rt_feature * 10 ** 1) / 10 ** 1
                    #print(rts)
                    mz_feature = float(df['mz_values'][i])
                    mz_feature_rounded = round(mz_feature, 1)
                    mz_feature_one_dec = math.floor(mz_feature * 10 ** 1) / 10 ** 1
                    #print(mzs)
                    if abs(rt_sample - rt_feature) <= rt_window:
                        if abs(mz_sample - mz_feature) <= mz_window:
                            sample_name = sample[:-4]
                            dir_mgf = label + '/' + sample_name + '/'
                            print('rt', rt_sample, 'mz of sample:', mz_sample, 'mz of feature:', mz_feature, sample_name)
                            try:
                                os.makedirs(dir_mgf)
                            except:
                                pass
                            filename = label + sample_name + title[:30] + '.mgf'
                            fea_mgf_path = dir_mgf + filename
                            with open(fea_mgf_path, 'w') as d:
                                None
                            for r in lb:
                                with open(fea_mgf_path, 'a') as f:
                                    f.write(r)
                            #write to results.csv file
                            data_count = data_count + 1
                            data[data_count] = {"Label": label, "Sample": sample_name, 'MZ of Sample': mz_sample, 'MZ of Feature': mz_feature, 'Precursor Ion Intensity in Sample': precursor_ion_intensity, 'Difference': (mz_sample - mz_feature), 'RT of Sample': rt_sample, 'RT of Feature': rt_feature, 'Energy': energy, 'File Name': filename}                        
                lb = []
                
output = pd.DataFrame(data=data).T
output.to_csv("Results.csv")

for i in range(len(df['labels'])):
    label = df['labels'][i]
    if label not in output['Label'].tolist():
        print('Did not find', label)